In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

import gm2
import trfp
import plotting_functions as plt2
# import analysis_helper as helper
# import helper_function_candidates as helper_old
# import df_analysis_funcs as df_func
import helper_functions as helper

import field_map_config_run2 as map_config

from IPython.display import clear_output

blinds = np.loadtxt('blinds.txt')
pair_dict = map_config.pair_dict

def azi_avg(moment_df):
    
    weight = np.diff(trfp.geometry.STATION_BARCODE_EDGES)
    weight[2] = weight[2]+360.
    weight = weight/360.
    
    avg_df = pd.DataFrame(index=moment_df.index)
    for m in range(9):
        stms = ['st'+str(st)+',m'+str(m+1) for st in range(72)]
        avg_df['m'+str(m+1)] = moment_df[stms].multiply(weight).sum(axis=1)
    return avg_df


Welcome to JupyROOT 6.22/02


In [3]:
runs = map_config.runs

columns = [['st'+str(st)+',dt'] +  ['st'+str(st)+',m'+str(m+1) for m in range(9)] for st in range(72)]
columns = [item for sublist in columns for item in sublist]
sync_offset_df = pd.DataFrame(columns=columns)

for run in runs:
    filename = 'hdf5/2021-01-11_run_' + run + '.h5'

    interp_dfs, keys, subrun_df = helper.read_dfs(filename)
    run_pair_dict = pair_dict[run]

    moment_dfs = helper.interp_to_moment(interp_dfs, keys)
    moment_dfs = helper.blind_moments(moment_dfs, keys, blinds)
    corrected_dfs = helper.moment_to_corrected(moment_dfs, keys)

#     baselines = helper.station_average(corrected_dfs, keys)

    ## for endgame only
    # for item in baselines:
    #     baselines[item]['tr_df_9'] = baselines[item]['tr_df_8']
    # baselines['time']['tr_df_9'] = baselines['time']['tr_df_7']

#     vtm_dfs = helper.calculate_vtms(corrected_dfs, keys, baselines, run_pair_dict)

    bloch_style_dfs = helper.bloch_style_moments(corrected_dfs, keys)
    baselines_bloch = helper.station_average(bloch_style_dfs, keys)

    ## for endgame only
    # for item in baselines_bloch:
    #     baselines_bloch[item]['tr_df_9'] = baselines_bloch[item]['tr_df_8']
    # baselines_bloch['time']['tr_df_9'] = baselines_bloch['time']['tr_df_7']

    vtm_dfs_bloch = helper.calculate_vtms(bloch_style_dfs, keys, baselines_bloch, run_pair_dict)
    
    
    
    ## Sync Offset

    
    
    for pair in pair_dict[run]:
        tr_bl_1 = baselines_bloch['tr'][pair_dict[run][pair][0]]
        tr_bl_2 = baselines_bloch['tr'][pair_dict[run][pair][1]]
        fp_bl_1 = baselines_bloch['fp'][pair_dict[run][pair][0]]
        fp_bl_2 = baselines_bloch['fp'][pair_dict[run][pair][1]]
        time_bl_1 = baselines_bloch['time'][pair_dict[run][pair][0]]
        time_bl_2 = baselines_bloch['time'][pair_dict[run][pair][1]]
        
        delta_tr = tr_bl_2 - tr_bl_1
        delta_fp = fp_bl_2 - fp_bl_1
        delta_time = time_bl_2 - time_bl_1
        
        sync_offsets = np.empty((72,9))
        for st in range(72):
            J = helper._choose_J(st)
            sync_offsets[st,0:5] = delta_tr[st,0:5] -  np.matmul(J, delta_fp[st,0:5])
            sync_offsets[st,5:9] = delta_tr[st,5:9]
        
        row = {}
        for st in range(72):
            for m in range(9):
                row['st'+str(st)+',m'+str(m+1)] = sync_offsets[st,m]
            row['st'+str(st)+',dt'] = delta_time[st]

        sync_offset_df = sync_offset_df.append(pd.DataFrame(row, index=['run_'+run+pair[6]]))
    
    
    ## Done w/ sync offsets

    clear_output()
    
    for key in vtm_dfs_bloch.keys():
#         a = vtm_dfs[key].index.values
#         bins = np.array([a,a]).T.flatten()[:len(a)]

#         vtm_2_sec_df = vtm_dfs_bloch[key].groupby(bins).mean()
#         vtm_2_sec_df = vtm_2_sec_df.set_index((a[::2]+0.5))

#         weight = trfp.STATION_BARCODE_EDGES[1:] - trfp.STATION_BARCODE_EDGES[:-1]
#         weight[2] = weight[2] + 360

#         vtm_2_sec_azi_avg_df = pd.DataFrame(index=vtm_2_sec_df.index)

#         for m in range(9):
#             stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
#             vtm_2_sec_azi_avg_df['m'+str(m+1)] = vtm_2_sec_df[stm_list].multiply(weight).sum(axis=1)/360
            
        # need to save 4 data frames

        save_key = 'run_' + run + key[6]
        save_dir = '/data2/aetb/'
        save_prefix = '2021-03-17_run-2_'
        
        print 'Saving ' + save_key

        # save purcell style

#         save_path = save_dir + save_prefix + 'purcell_maps.h5'
#         vtm_dfs[key].to_hdf(save_path, key=save_key)

        # save hybrid

        save_path = save_dir + save_prefix + 'hybrid_maps.h5'
        vtm_dfs_bloch[key].to_hdf(save_path, key=save_key)

        # save 2 sec hybrid

#         save_path = save_dir + save_prefix + 'hybrid_maps_2sec.h5'
#         vtm_2_sec_df.to_hdf(save_path, key=save_key)

        # save 2 sec azi avg hybrid

#         save_path = save_dir + save_prefix + 'hybrid_maps_2sec_aziavg.h5'
#         vtm_2_sec_azi_avg_df.to_hdf(save_path, key=save_key)

print 'Saving Sync Offsets'

save_path = save_dir + save_prefix + 'hybrid_syncs.h5'
save_key = 'sync_offset_df'
sync_offset_df.to_hdf(save_path, key=save_key)

Saving run_2b1
Saving run_2b2

Reading fp_df_1

Reading fp_df_2

Reading fp_df_3

Reading fp_df_4

Reading tr_df_1

Reading tr_df_2

Reading tr_df_3a

Reading tr_df_3b

Reading tr_df_4

Reading tr_df_5

Calculating moments for fp_df_1
Finished calculating all moments for 361491 events.          

Calculating moments for fp_df_2
Finished calculating all moments for 260932 events.     

Calculating moments for fp_df_3
Finished calculating all moments for 263196 events.    

Calculating moments for fp_df_4
Finished calculating all moments for 197469 events.

Calculating moments for tr_df_1
Finished calculating all moments for 4366 events.

Calculating moments for tr_df_2
Finished calculating all moments for 4319 events.

Calculating moments for tr_df_3a
Finished calculating all moments for 208 events.

Calculating moments for tr_df_3b
Finished calculating all moments for 4365 events.

Calculating moments for tr_df_4
Finished calculating all moments for 4389 events.

Calculating moments fo

TypeError: expected non-empty vector for x

In [ ]:
sync_offset_df.head()
plt.hist(sync_offset_df[['st'+str(st)+',m1' for st in range(72)]].iloc[0])

In [ ]:
time_bl_1

In [ ]:
def sync_offset_calc(tr_corr_df_1, tr_corr_df_2):
    tr_baseline_1, fp_baseline_1, baseline_time_1, _, _ = trolley_run_station_average(tr_corr_df_1)
    tr_baseline_2, fp_baseline_2, baseline_time_2, _, _ = trolley_run_station_average(tr_corr_df_2)
    delta_time = baseline_time_2 - baseline_time_1
    
    delta_tr_baseline = tr_baseline_2 - tr_baseline_1
    delta_fp_baseline = fp_baseline_2 - fp_baseline_1
    
    sync_offsets = np.empty((72,9))
    for st in range(72):
        J = _choose_J(st)
        sync_offsets[st,0:5] = delta_tr_baseline[st,0:5] -  np.matmul(J, delta_fp_baseline[st,0:5])
        sync_offsets[st,5:9] = delta_tr_baseline[st,5:9]
    
    return sync_offsets, delta_time

In [ ]:
df = pd.read_hdf('/data2/aetb/2021-03-17_hybrid_maps.h5', key='run_2a1')
avg_df = azi_avg(df)
plt.plot(avg_df.index.values, avg_df['m2'], '.')

In [ ]:
df = pd.read_hdf('/data2/aetb/2021-03-17_hybrid_maps.h5', key='run_2b1')
avg_df = azi_avg(df)
plt.plot(avg_df.index.values, avg_df['m2'], '.')

In [ ]:
df = pd.read_hdf('/data2/aetb/2021-03-17_hybrid_maps.h5', key='run_2b2')
avg_df = azi_avg(df)
plt.plot(avg_df.index.values, avg_df['m2'], '.')


In [ ]:
## looking into reducing df sizes

avg_df.info()

In [ ]:
x = avg_df['m1'].iloc[0]
type(x)

In [ ]:
print x
print x.astype(np.float64)
print x.astype(np.float32)
print x.astype(np.float16)

In [ ]:
print (x + 61.79e6).astype(np.float64)
print (x + 61.79e6).astype(np.float32)
print (x + 61.79e6).astype(np.float16)

In [ ]:
print np.binary_repr(61840884)
print np.binary_repr(61840884)
print len(np.binary_repr(61840884))